In [1]:
# %run initialize/1_coastal_classification.ipynb

# Characterization of Coastal Systems

Welcome to the first notebook exercise of Coastal Systems (TU Delft, MSc Coastal Engineering)! This is the first year that we will experiment with notebooks in this course. With these notebooks we hope to provide you with interactive course material that helps you better understand the processes and concepts that we teach in this course. Please let us know how you find the notebooks - we appreciate your feedback!

Chapter 2 of [Coastal Dynamics Open Textbook](https://textbooks.open.tudelft.nl/textbooks/catalog/view/37/92/383-1) describes the large geographical variation of coasts across the world. It explains how the coasts that we have today are shaped by both present-day processes and processes millions years ago. It distinguishes between three different order of features, which are associated to different orders of of time. In this notebook we will look at coastal systems at these different orders of scale. 

## Import libraries that we use for our analysis

In the two cells below we import the libraries that we need for the analysis. We also set some path settings to load the data and source code. For example, in the cell below we add the `src` directory to the system path, which allows us to import generic functions from `../../src/coastpy`. 

In [1]:
import pathlib
import sys

import colorcet as cc
import dask.dataframe as dd
import geopandas as gpd
import numpy as np
import panel as pn
import holoviews as hv
import hvplot.pandas  # noqa: API import
from bokeh.models import PanTool, WheelZoomTool

print("Packages succesfully loaded")

c:\Users\khdeb\miniforge-pypy3\envs\coastal\Lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 10.0.1. Please consider upgrading.
  warnings.warn(


Packages succesfully loaded


In [2]:
# THESE PACKAGES ARE NO LONGER USED

# import os

# import hvplot.xarray  # noqa: API import
# import ipyleaflet
# import pandas as pd
# import matplotlib.pyplot as plt
# import matplotlib.animation as animation
# from geoviews import tile_sources as gvts
# from ipyleaflet import Map, Marker, ScaleControl, basemaps
# import ipywidgets as widgets
# from ipywidgets import HTML, interact, fixed, interact_manual, interactive
# import IPython
# from IPython.display import HTML, display
# from random import shuffle, uniform
# from coastal_dynamics.geometries import geo_bbox
# from PIL import Image

In [3]:
# Set project directory
cwd = pathlib.Path().resolve()
proj_dir = cwd.parent  # this is the root of the CoastalCodeBook
sys.path.append(str(proj_dir / "src"))

DATA_DIR = proj_dir / "data"
coastal_systems_fp = DATA_DIR / "01_coastal_systems.gpkg"

In [4]:
# Set up the questions
import coastal_dynamics as cd
from coastal_dynamics.io import load_questions

# Activate the Panel widget to make the visualizations.
pn.extension()

QUESTION_DIR = pathlib.Path.cwd().parent / "notebooks" / "questions"
question_fp = QUESTION_DIR / "1_coastal_classification.json"

questions = load_questions(question_fp)

## Part 1: Tectonic classification

In this notebook we will start with the broadest (or first order) features of the coast that cover large geographical distances (thousands of kilometres) and are linked to the long-term geological process of plate tectonics. We will do so by using earthquake data from the [USGS](https://earthquake.usgs.gov/earthquakes/search/). The dataset we load contains a sample (10\%) of observed eartquakes between Jan 2000 and Dec 2018. Why earthquake data? Earthquake data reveals geologists the mysteries of the deep, but also for coastal researchers the data is insightful. Let's first load the data by running the next cells. 

### Load the earthquake data

We load the data (tabular data including geometries) and index the columns to only keep the data in memory that we actually need. In total the dataset contains 2.2 million earthquakes, but here we use a sample (10\%), so the data contains approx. 220k eartquake entries. If you find that the interactive panel responds slow to how you adjust the widgets, please consider to take another sample. You can do so by uncommenting the sample line in the next cell. So if you set frac=0.1 you have a dataframe with approx. 22k unique earthquakes over the world. 
 

In [5]:
WEB_MERCATOR_LIMITS = (
    -20037508.342789244,
    20037508.342789244,
)  # max polar latitudes that can be handled in World Mercator

df = (
    dd.read_parquet(DATA_DIR / "01_earthquakes_sample.parquet")
    .sample(
        frac=0.1
    )  # uncomment this line if loading the data takes too long on your computer
    .set_index("time")
    .compute()
    .tz_localize(None)
    .sort_index()
)


# To save memory we drop most of the columns. Also we drop the polar latitudes that cannot be displayed in the web mercator projection.
df = df[["mag", "depth", "latitude", "longitude", "place", "type"]][
    df["northing"] < WEB_MERCATOR_LIMITS[1]
]
# df.head()

In [8]:
# Let's get data for the -200m isobaths!

data200 = gpd.read_file(str(proj_dir / "data/01_coastal_systems_isobaths200.gpkg"))

In [9]:
# To make plotting a bit faster, we only use isobaths longer than a certain length. We therefore first
# have to project to a new coordinate system to get the lengths of the isobaths, and use that as a
# mask to select the isobaths we want to keep
data200['length'] = data200.to_crs('EPSG:3857').geometry.length
data200 = data200[data200['length']>5*10**6]

### Visualization of the earthquake data

To explore the data we use visualization tools from the [Holoviz project](https://holoviz.org/) that makes high-level tools to simplify visualization in Python. In the next cell we enable the interactive mode on the data dataframe, create widgets to explore the data and filter the dataframe accordingly. To explore the eartquake data we create an overlay of the eartquakes on a tileset of ESRI Imagery. Please note that the code in the next cell will only do the computations and store the result in an object called `panel`. To actually see the results you have to run one more cell; the one that calls this object panel. 



In [14]:
title_bar =  pn.pane.Markdown(
        "## Part 1: Tectonic classification",
        styles={"color": "black"},
        width=400,
        # margin=(10, 5, 10, 15),
    )

# define widgets that can be used to index the data
magnitude_slider = pn.widgets.RangeSlider(
    name="Earthquake magnitude [Richter]", start=0.1, end=10
)
depth_slider = pn.widgets.RangeSlider(name="Earthquake depth [km]", start=0.1, end=650)
date_slider = pn.widgets.DateRangeSlider(
    name="Date", start=df.index[0], end=df.index[-1]
)
column_types = pn.widgets.Select(name='Show earthquake magnitude or depth?', options=["mag", "depth"])

plot_isobaths = pn.widgets.Select(name='Plot isobaths -200m?', options=['no', 'yes'])


@pn.depends(
    magnitude_slider.param.value_start,
    magnitude_slider.param.value_end,
    depth_slider.param.value_start,
    depth_slider.param.value_end,
    date_slider.param.value_start,
    date_slider.param.value_end,
    column_types.param.value,
    plot_isobaths.param.value,
)
def plot_earthquake_panel(
    magnitude_start,
    magnitude_end,
    depth_start,
    depth_end,
    date_start,
    date_end,
    column_type,
    plot_isobath
):
    panel = df[
        (df.mag > magnitude_start)
        & (df.mag < magnitude_end)
        & (df.depth > depth_start)
        & (df.depth < depth_end)
        & (df.index >= date_start)
        & (df.index <= date_end)
    ]
    # inverted fire colormap from colorcet
    cmap = cc.CET_L4[::-1]
    colorbar_labels = {"mag": "Magnitude [Richter]", "depth": "Earthquake depth [km]"}

    p = panel.hvplot.points(
        x="longitude",
        y="latitude",
        geo=True,
        color=column_type,
        global_extent=True,
        tiles="ESRI",
        # frame_width=900,
        ylabel="Latitude [deg]",
        xlabel="Longitude [deg]",
        cmap=cmap,
        tools=["tap"],
        hover_cols=["place", "time"],
        logz=True,
        clim=(1, None),
        clabel=colorbar_labels[column_type],
    )

    if plot_isobath=='yes':
        baths = data200.hvplot(geo=True, line_width=2, line_color='white', line_dash='dashed')
        p = p*baths

    p.opts(width=1000, height=500, tools=["wheel_zoom"])

    return p


earthquake_panel = pn.Column(pn.Row(
    pn.Column(
    pn.Row(title_bar, align='start'),
    pn.Row(plot_isobaths, align="start"),
    pn.Row(column_types, align="start"),
    ),
    pn.Column(
    pn.Row(magnitude_slider, align="start"),
    pn.Row(depth_slider, align="start"),
    pn.Row(date_slider, align="start"),
    ), pn.Column()
    ),
    pn.Row(plot_earthquake_panel, align="center"),
)

In [15]:
# change value of 'plot_where' to:
# 'inline' if you would like the plot to show in the notebook
# 'pop-out' if you would like the plot to show in a new tab (i.e. seperate window)

plot_where = "pop-out"

**If the visualization is too slow, please follow the instructions in loading the data for taking a sample.**

After running the cell below you will have a panel with several widgets to index the eartquake data; by magnitude, depth and time, while the colors on the map show either the magintude or the depth of the earthquakes. 

In [16]:
def show_earthquake(plot_where):
    if plot_where == "inline":
        return earthquake_panel
    elif plot_where == "pop-out":
        earthquake_panel.show()
    else:
        print("please use either inline or pop-out for the plot_where variable")


show_earthquake(plot_where)

Launching server at http://localhost:61512


### Explore the earthquake data & questions
Using the earthquake plot, try to answer the questions below. In order to view the questions, run the code cell.

In [ ]:
q1 = cd.QuestionFactory(questions["Q1-1"]).serve()
q2 = cd.QuestionFactory(questions["Q1-2"]).serve()
q3 = cd.QuestionFactory(questions["Q1-3"]).serve()
q4 = cd.QuestionFactory(questions["Q1-4"]).serve()


pn.Column(q1, q2, q3, q4)

TypeError: standalone_docs_json_and_render_items() takes 1 positional argument but 2 were given

Column
    [0] Column
        [0] StaticText(name='Q1-1', value='At what depth d...)
        [1] RadioBoxGroup(name='Options', options=['less than 33 km', ...], value='less than 33 km')
        [2] Button(name='Submit')
        [3] StaticText()
    [1] Column
        [0] StaticText(name='Q1-2', value='Where do earthquakes w...)
        [1] CheckBoxGroup(name='Options', options=['wherever two oceanic pla...])
        [2] Button(name='Submit')
        [3] StaticText()
    [2] Column
        [0] StaticText(name='Q1-3', value='How should one s...)
        [1] RadioBoxGroup(name='Options', options=['0 - 33 km', ...], value='0 - 33 km')
        [2] Button(name='Submit')
        [3] StaticText()
    [3] Column
        [0] StaticText(name='Q1-4', value='How can one d...)
        [1] CheckBoxGroup(name='Options', options=['width of earthquake zone...])
        [2] Button(name='Submit')
        [3] StaticText()

Earthquake data support one of the most fundamental processes in the geology: plate tectonics. Although plate tectonics is a relatively slow process that acts on the geological time scale, it has had an enormous impact on the formation of coastlines and determines the broadest features of the coast.

In [ ]:
q5 = cd.QuestionFactory(questions["Q1-5"]).serve()

pn.Column(q5)

For the upcoming questions, consider the east Australian "Gold Coast".

In [ ]:
q6 = cd.QuestionFactory(questions["Q1-6"]).serve()
q7 = cd.QuestionFactory(questions["Q1-7"]).serve()
q8 = cd.QuestionFactory(questions["Q1-8"]).serve()
q9 = cd.QuestionFactory(questions["Q1-9"]).serve()

pn.Column(q6, q7, q8, q9)

In 1971 Inman, D. L. & Nordstrom, C. E. used plate tectonics to classify the coast. They distinguish three main types of coasts: leading edge, trailing edge and marginal sea. In the questions below, match the correct characteristics to each class.

In [ ]:
q10 = cd.QuestionFactory(questions["Q1-10"]).serve()
q11 = cd.QuestionFactory(questions["Q1-11"]).serve()
q12 = cd.QuestionFactory(questions["Q1-12"]).serve()

pn.Column(q10, q11, q12)

Now that we have looked at the typical characteristics of each coastal type, we will look at some examples. Each question below specifies a different location. Use the earthquake panel to zoom to the locations specified, and classify the coasts to the correct classes. 

In [ ]:
q13 = cd.QuestionFactory(questions["Q1-13"]).serve()
q14 = cd.QuestionFactory(questions["Q1-14"]).serve()
q15 = cd.QuestionFactory(questions["Q1-15"]).serve()
q16 = cd.QuestionFactory(questions["Q1-16"]).serve()

pn.Column(q13, q14, q15, q16)

Inman, D. L. & Nordstrom (1971) further distinguish Afro-trailing-edge coasts and Amero-trailing-edge coasts based on differences in sediment supplies.

In [ ]:
q17 = cd.QuestionFactory(questions["Q1-17"]).serve()
q18 = cd.QuestionFactory(questions["Q1-18"]).serve()
q19 = cd.QuestionFactory(questions["Q1-19"]).serve()

pn.Column(q17, q18, q19)

####################################################

TO BE REMOVED

1) How do the earthquake magnitude and earthquake depth relate to the coasts that we see? (Hint: See Figure 2.3 in the textbook and consider how deep under the ground the plates are moving. Extra hint: How do earthquake magnitude and depth differ for convergent and divergent plate boundaries?)

2) Earthquake data support one of the most fundamental processes in the geology: plate tectonics. Although plate tectonics is a relatively slow process that acts on the [geological time scale](https://cdn.britannica.com/67/73167-050-B9A74092/chart.jpg), it has had an enormous impact on the formation of coastlines and determines the broadest features of the coast. What are two important inherited aspects of this process? (Hint: see Figure 2.10 and Sec. 2.3.3 in the textbook.) 

3) In 1971 Inman, D. L. & Nordstrom, C. E. used plate tectonics to classify the coast. Explain the classification that they introduced. What are the three different classes that they distinguish? How do they match with the earthquake data as you can explore in the panel? 

4) Can you identify or predict areas around the world where you will find the coasts that are distinguished by Inman, D. L. & Nordstrom, C. E.? For instance, what kind of coasts do you have in Chili? And how are they different to the east coast of the USA? And what is characteristic about the East China sea? 

5) Inman, D. L. & Nordstrom (1971) further distinguish Afro-trailing-edge coasts and Amero-trailing-edge coasts based on differences in sediment supplies. What is the main cause of these differences in sediment supply? And how do you expect the differences in sediment input to show in the coastal geomorphology?
####################################################

## Part 2: Process-based coastal classification

In the section part of this notebook we will explore several coastal systems around the world considering the second and third order scale of features. In chapter two of the Coastal Dynamics open textbook it is explained how coastal systems can be classified according to the processes that characterize these systems. For example, one of the figures (below) shows how the relative influence of fluvial, wave and tidal processes influences the shape of coastal features. The idea of this exercise is that you identify the signatures of the processes that are introduced in chapter 2 in several coastal systems around the world. 

![image](./figures/01_coastal_forcing.png)

### The coastal systems data

In the cell below we define a small plot function that generates a ESRI World Imagery basemap given a longitude, latitude, zoom level and name. Also, a small sheet of coastal systems around the world is loaded into `geopandas`, a Python library for geospatial tabular data. In the cells afterwards we sample this dataframe and show the coastal system on a map. Since the sample is random you might encounter the same coastal system multiple times; then you can just run the cell again to get another 'coastal draw'.  


In [17]:
coastal_systems = gpd.read_file(coastal_systems_fp)

In [21]:
title_bar = pn.Row(
    pn.pane.Markdown(
        "## Exercise 2: Coastal system characterization",
        styles={"color": "black"},
        width=800,
        sizing_mode="fixed",
        margin=(10, 5, 10, 15),
    ),
    pn.Spacer(),
)

options = coastal_systems.name.to_list()
coastal_systems_slider = pn.widgets.Select(
    name="Coastal system", options=options, value=np.random.choice(options)
)

plot_isobaths = pn.widgets.Select(name='Plot isobaths -200m?', options=['no', 'yes'])

@pn.depends(coastal_systems_slider.param.value, plot_isobaths.param.value)
def plot_coastal_system(name, plot_isobath):
    system = coastal_systems.loc[coastal_systems["name"] == name].copy()
    west, south, east, north = system[
        ["west", "south", "east", "north"]
    ].values.flatten()

    p = system.hvplot.points(
        x="lon",
        y="lat",
        geo=True,
        color="red",
        alpha=0,
        xlim=(west, east),
        ylim=(south, north),
        tiles="ESRI",
        frame_width=1100,
        ylabel="Latitude [deg]",
        xlabel="Longitude [deg]",
    )

    if plot_isobath=='yes':
        baths = data200.hvplot(geo=True, line_width=2, line_color='white', line_dash='dashed')
        p = p*baths

    p.opts(frame_width=1000, frame_height=500, tools=["pan", "wheel_zoom"])

    return p


app = pn.Column(
    # title_bar,
    pn.Row(plot_isobaths, align="center"),
    pn.Row(coastal_systems_slider, align="center"),
    pn.Row(plot_coastal_system, align="center"),
)

After running the cell below you will have a panel showing sattelite images of different coastal systems.

In [22]:
# change value of 'plot_where' to:
# 'inline' if you would like the plot to show in the notebook
# 'pop-out' if you would like the plot to show in a new tab (i.e. seperate window)

plot_where = "pop-out"

In [ ]:
def show_coastal_system(plot_where):
    if plot_where == "inline":
        return app
    elif plot_where == "pop-out":
        app.show()
    else:
        print("please use either 'inline' or 'pop-out' for the 'plot_where' variable")


show_coastal_system(plot_where="pop-out")

Launching server at http://localhost:61657


Using this panel, try to answer the questions below. The questions are seperated by 'theory' and 'application'.

### Questions 2a: theory
Running the cell below will generate theory questions. Use the sattelite images of different coastal systems to answer the questions.

In [ ]:
# Questions 2a
q20 = cd.QuestionFactory(questions["Q1-20"]).serve()
q21 = cd.QuestionFactory(questions["Q1-21"]).serve()
q22 = cd.QuestionFactory(questions["Q1-22"]).serve()
q23 = cd.QuestionFactory(questions["Q1-23"]).serve()
q24 = cd.QuestionFactory(questions["Q1-24"]).serve()
q25 = cd.QuestionFactory(questions["Q1-25"]).serve()
q26 = cd.QuestionFactory(questions["Q1-26"]).serve()
q27 = cd.QuestionFactory(questions["Q1-27"]).serve()
q28 = cd.QuestionFactory(questions["Q1-28"]).serve()
q29 = cd.QuestionFactory(questions["Q1-29"]).serve()
q30 = cd.QuestionFactory(questions["Q1-30"]).serve()
q31 = cd.QuestionFactory(questions["Q1-31"]).serve()
q32 = cd.QuestionFactory(questions["Q1-32"]).serve()
q33 = cd.QuestionFactory(questions["Q1-33"]).serve()

pn.Column(q20, q21, q22, q23, q24, q25, q26, q27, q28, q29, q30, q31, q32, q33)

### Questions 2b: application
Running the cell below will generate application questions. Use the sattelite images of different coastal systems to answer the questions.

In [ ]:
# Questions 2b
q34 = cd.QuestionFactory(questions["Q1-34"]).serve()
q35 = cd.QuestionFactory(questions["Q1-35"]).serve()
q36 = cd.QuestionFactory(questions["Q1-36"]).serve()
q37 = cd.QuestionFactory(questions["Q1-37"]).serve()
q38 = cd.QuestionFactory(questions["Q1-38"]).serve()
q39 = cd.QuestionFactory(questions["Q1-39"]).serve()
q40 = cd.QuestionFactory(questions["Q1-40"]).serve()
q41 = cd.QuestionFactory(questions["Q1-41"]).serve()
q42 = cd.QuestionFactory(questions["Q1-42"]).serve()
q43 = cd.QuestionFactory(questions["Q1-43"]).serve()
q44 = cd.QuestionFactory(questions["Q1-44"]).serve()
q45 = cd.QuestionFactory(questions["Q1-45"]).serve()
q46 = cd.QuestionFactory(questions["Q1-46"]).serve()
q47 = cd.QuestionFactory(questions["Q1-47"]).serve()

pn.Column(q34, q35, q36, q37, q38, q39, q40, q41, q42, q43, q44, q45, q46, q47)

This is the end of the notebook for week 1. We discussed tectonic and process-based classification of coastal systems. For further reading, see chapter 2 of the book. We hope you had fun!

############################################################################

TO BE REMOVED

### Explore the coastal systems

While sampling over a range of coastal systems, try to answer the following questions. 

1. Find and compare a heavily engineered river-dominated delta and a more natural river-dominated delta
2. Compare the scale of the biggest and smallest tidal basin in the dataset
3. Find the estuarine and deltaic systems with a spit
4. Compare and contrast wave-dominated deltas with high and low sediment supply. How can you tell?
5. Find a tidal estuary with large fine (muddy) sediment supply, then find one with a large coarse (sandy) sediment supply. How can you tell the difference?
6. The eastern and western tips of the Dutch and German Wadden Islands are very different beach ridge environments. How might differences in sediment supply explain this? Where is the sediment coming from?
7. The Dune du Pilat in France is one of the world's largest coastal sand dunes (it is also one of the coolest and you should definitely visit if you get the chance!). Why is it located on the east side of Arcachon Inlet and not the west?
8. Look at the northern Jiangsu coast in China. What might explain the limited sediment supply in this location?
9. Find an estuary or tidal bay with extensive intertidal flats. Do you see salt marshes or mangrove forests nearby?  Why or why not?
10. Find an inlet with jetties. How might this affect the way it evolves?
11. Find a delta/estuary/inlet whose shape is constrained by the presence of rocky coastal features.  
12. The Albufeira Lagoon in Portugal opens and closes seasonally. In the image shown, is it open or closed? When and how might it open or close? 
13. Find examples of heavily urbanized estuaries. How might these human interventions influence the natural processes there?
14. Based on these satellite images, which is the most beautiful site? Taking a moment to appreciate the beauty of these natural systems is an important part of your job as coastal engineers.

############################################################################